Exercise: Augmenting the LSTM part-of-speech tagger with character-level features ~~~~~~~~~~~~~~~~~

In the example above, each word had an embedding, which served as the inputs to our sequence model. Let's augment the word embeddings with a representation derived from the characters of the word. We expect that this should help significantly, since character-level information like affixes have a large bearing on part-of-speech. For example, words with the affix -ly are almost always tagged as adverbs in English.

To do this, let  𝑐𝑤  be the character-level representation of word  𝑤 . Let  𝑥𝑤  be the word embedding as before. Then the input to our sequence model is the concatenation of  𝑥𝑤  and  𝑐𝑤 . So if  𝑥𝑤  has dimension 5, and  𝑐𝑤  dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the characters of a word, and let  𝑐𝑤  be the final hidden state of this LSTM. Hints:

There are going to be two LSTM's in your new model. The original one that outputs POS tag scores, and the new one that outputs a character-level representation of each word.
To do a sequence model over characters, you will have to embed characters. The character embeddings will be the input to the character LSTM.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [6]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

WORD_EMBEDDING_DIM = 6
CHAR_EMBEDDING_DIM = 3
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [7]:
#define the model
class LSTMTagger(nn.Module):

    def __init__(self, word_embedding_dim, char_embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTagger, self).__init__()
        
        self.word_embeddings = nn.Embedding(vocab_size, word_embedding_dim)
        
        self.lstm1=nn.LSTM(word_embedding_dim,char_embedding_dim)
        
        self.lstm2=nn.LSTM(word_embedding_dim+char_embedding_dim,hidden_dim)
        
        self.hidden2tag=nn.Linear(hidden_dim,target_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence) #embedding
        embeds=embeds.view(len(sentence), 1, -1) #转为lstm的输入格式,三维tensor
        
        lstm_out, hidden = self.lstm1(embeds) #经过lstm层
        lstm2_input=torch.cat((embeds,lstm_out),2) #拼接
        
        lstm_out,_ =self.lstm2(lstm2_input) #再经过lstm层
        lstm_out=lstm_out.view(len(sentence), -1) #转为linear层的输入格式,由于是一次性计算,所以是一个二维tensor
        
        tag_space = self.hidden2tag(lstm_out) #经过linear层
        tag_scores = F.log_softmax(tag_space, dim=1) #输出
        return tag_scores

In [8]:
model = LSTMTagger(WORD_EMBEDDING_DIM, CHAR_EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    print(inputs.shape)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([0, 1, 2, 3, 4])
torch.Size([5])
tensor([[-1.4820, -0.8894, -1.0163],
        [-1.5445, -0.8368, -1.0398],
        [-1.4893, -0.9318, -0.9660],
        [-1.4020, -0.9443, -1.0081],
        [-1.3395, -0.9498, -1.0464]])
tensor([[-0.5781, -1.1251, -2.1678],
        [-3.3083, -0.0592, -3.8658],
        [-2.7158, -4.9142, -0.0763],
        [-0.0553, -3.3075, -4.0615],
        [-2.0891, -0.1384, -5.2061]])
